In [21]:
# from tensorflow.keras.layers.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import InputLayer, Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
# from skkeras import KerasClassifier
from scikeras.wrappers import KerasClassifier
from keras.models import Sequential 
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
import glob
import os


In [22]:
# npz data to pd.
def npz_to_csv(file_path):
    file = np.load(file_path, allow_pickle=True)
    X_df = pd.DataFrame(file['X'])
    y_df = pd.DataFrame(file['y'])
    return X_df, y_df

In [23]:
param_dist = {
    'units': [50, 100, 150],
    'dropout_rate': [0.1, 0.2, 0.3],
    'batch_size': [16, 32, 64],
    'epochs': [10, 20, 30],
    'learning_rate' : [0.001, 0.003, 0.005, 0.007, 0.01]
}


In [24]:
from tensorflow.keras.optimizers import legacy
def create_model():
    model = Sequential()
    model.add(LSTM(units=units, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1, activation='sigmoid'))
    optimizer = legacy.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model


val losses와 val acc를 한 번에 plotting하는 함수

In [25]:
def plot_validation_metrics(val_losses, val_accs):
    num_folds = len(val_losses)
    
    folds = range(1, num_folds + 1)
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6), sharex=True)
    
    ax1.plot(folds, val_losses, marker='o', linestyle='-', color='b')
    ax1.set_ylabel('Validation Loss')
    ax1.set_title('Validation Loss and Accuracy')
    ax1.grid(True)
    
    ax2.plot(folds, val_accs, marker='o', linestyle='-', color='g')
    ax2.set_xlabel('Fold')
    ax2.set_ylabel('Validation Accuracy')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

In [26]:
# file_path = '/path/to/Classical' 각자가 저장한 폴더로 연결해주세요
file_path = './datasets/Classical/*.npz'
file_list = glob.glob(file_path)

폴더안에 자동으로 학습하고 기록하고싶은 데이터만 넣어두고, 이 밑에 코드 돌려두세요.

In [27]:
proper_datasets = []

In [28]:
for file in file_list:
    _, y = npz_to_csv(file)
    if y[y[0] ==1].size / y.size > 0.1:
        proper_datasets.append(file)

print(proper_datasets)

['./datasets/Classical/45_SWAT.npz', './datasets/Classical/65_MachineryFault.npz', './datasets/Classical/43_Motorcondition2.npz', './datasets/Classical/54_CNCMachining.npz', './datasets/Classical/42_Motorcondition1.npz', './datasets/Classical/50_PLAID.npz', './datasets/Classical/48_Ladlefurnace.npz', './datasets/Classical/35_IMS.npz', './datasets/Classical/66_Cuttingblade.npz', './datasets/Classical/46_Sm4Tankbatch.npz', './datasets/Classical/61_Biopharmaceutical.npz', './datasets/Classical/58_MiningProcess.npz', './datasets/Classical/47_FordB_anreal.npz', './datasets/Classical/53_Walk2D.npz']


In [29]:
X, y = npz_to_csv(proper_datasets[0])

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


In [30]:
X_reshaped = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

In [31]:
X_reshaped.shape

(449919, 51, 1)

In [32]:
layers = [ [50, 100, 150], [0.1, 0.2, 0.3], [16, 32, 64], [10, 20, 30], [0.001, 0.003, 0.005, 0.007, 0.01] ]

In [33]:
model = KerasClassifier(
    model=create_model,
    input_shape=(X_reshaped.shape[0], X_reshaped[1],  1),
    verbose=0,
    shuffle=False,
    validation_split=0.2,
)

# RandomizedSearchCV 설정
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist,
                                   n_iter=10, scoring='accuracy', cv=3, verbose=1)


In [34]:
random_search_result = random_search.fit(X_reshaped, y)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


ValueError: Invalid parameter units for estimator KerasClassifier.
This issue can likely be resolved by setting this parameter in the KerasClassifier constructor:
`KerasClassifier(units=100)`
Check the list of available parameters with `estimator.get_params().keys()`

In [17]:
model.get_params()['param_dist']

KeyError: 'param_dist'

In [38]:
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from tensorflow.keras.optimizers import legacy


# 데이터 준비
# X_reshaped와 y를 준비합니다.
input_shape = (X_reshaped.shape[1], X_reshaped.shape[2])

# 모델 생성 함수
def create_model(units=100, dropout_rate=0.2, learning_rate=0.001):
    model = Sequential()
    model.add(LSTM(units=units, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1, activation='sigmoid'))
    optimizer = legacy.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# KerasClassifier 설정
model = KerasClassifier(model=create_model, verbose=0)

# 파라미터 분포 설정
param_dist = {
    'model__units': [50, 100, 150],
    'model__dropout_rate': [0.1, 0.2, 0.3],
    'batch_size': [16, 32, 64],
    'epochs': [10, 20, 30],
    'model__learning_rate': [0.001, 0.003, 0.005, 0.007, 0.01],
}

# RandomizedSearchCV 설정
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist,
                                    n_iter=10, scoring='accuracy', cv=3, verbose=1)

# 모델 학습
random_search.fit(X_reshaped, y)

# 결과 출력
print(f"Best Parameters: {random_search.best_params_}")
print(f"Best Score: {random_search.best_score_}")


Fitting 3 folds for each of 10 candidates, totalling 30 fits


ValueError: 
All the 30 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/seyeong/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/seyeong/anaconda3/lib/python3.10/site-packages/scikeras/wrappers.py", line 1501, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/Users/seyeong/anaconda3/lib/python3.10/site-packages/scikeras/wrappers.py", line 770, in fit
    self._fit(
  File "/Users/seyeong/anaconda3/lib/python3.10/site-packages/scikeras/wrappers.py", line 928, in _fit
    self._ensure_compiled_model()
  File "/Users/seyeong/anaconda3/lib/python3

In [42]:
from scikeras.wrappers import KerasClassifier
# from tensorflow.keras.models import Sequential
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from tensorflow.keras.optimizers import legacy

# 데이터 준비
# X_reshaped와 y를 준비합니다.
input_shape = (X_reshaped.shape[1], X_reshaped.shape[2])

# 모델 생성 함수
def create_model(units=100, dropout_rate=0.2, learning_rate=0.001):
    model = Sequential()
    model.add(LSTM(units=units, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1, activation='sigmoid'))
    optimizer = legacy.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# KerasClassifier 설정
model = KerasClassifier(model=create_model, verbose=0)

# 파라미터 분포 설정
param_dist = {
    'model__units': [50, 100, 150],
    'model__dropout_rate': [0.1, 0.2, 0.3],
    'batch_size': [16, 32, 64],
    'epochs': [10, 20, 30],
    'model__learning_rate': [0.001, 0.003, 0.005, 0.007, 0.01],
}

# RandomizedSearchCV 설정
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist,
                                    n_iter=10, scoring='accuracy', cv=3, verbose=1)

# 모델 학습
random_search.fit(X_reshaped, y)

# 결과 출력
print(f"Best Parameters: {random_search.best_params_}")
print(f"Best Score: {random_search.best_score_}")


Fitting 3 folds for each of 10 candidates, totalling 30 fits


ValueError: 
All the 30 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/seyeong/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/seyeong/anaconda3/lib/python3.10/site-packages/scikeras/wrappers.py", line 1501, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/Users/seyeong/anaconda3/lib/python3.10/site-packages/scikeras/wrappers.py", line 770, in fit
    self._fit(
  File "/Users/seyeong/anaconda3/lib/python3.10/site-packages/scikeras/wrappers.py", line 928, in _fit
    self._ensure_compiled_model()
  File "/Users/seyeong/anaconda3/lib/python3